# Segmenting and Clustering Neighborhoods in Toronto

1. Start by creating a new Notebook for this assignment.
1. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [8]:
!conda install -c conda-forge beautifulsoup4 --yes 

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge

beautifulsoup4 100% |################################| Time: 0:00:00  41.76 MB/s


In [1]:
import urllib.request as url
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

- using requests and beautifullsoup4 to download and parse the page

In [2]:
w_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
w_page = requests.get(w_url).text
soup = bs(w_page,'lxml')
#print(soup.prettify())

In [3]:

w_class='wikitable sortable'# jquery-tablesorter

rows = []
for w_row in soup.find('table',{'class':w_class}).find('tbody').find_all('tr'):
    row = [x.text.replace('\n','') for x in w_row.find_all('td')]
    if (len(row)==3):
        #print(row)
        rows.append(row)


df_postcode = pd.DataFrame(rows,columns = ['PostalCode','Borough','Neighbourhood'])
print(df_postcode.shape)
df_postcode.head(10)

(289, 3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


3. To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
df_filtered = df_postcode[df_postcode['Borough']!='Not assigned']
df_filtered.head(5)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
df_filtered[df_filtered['Borough']=='Queen\'s Park']

,PostalCode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [6]:
def check_neighbourhood(row):
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
    return row
df_filtered.apply(check_neighbourhood,axis=1)
df_filtered[df_filtered['Borough']=='Queen\'s Park']

,PostalCode,Borough,Neighbourhood
8,M7A,Queen's Park,Queen's Park


- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [7]:
df_grouped = pd.DataFrame(df_filtered.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(list)).reset_index()
df_grouped.head(5)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"[Rouge, Malvern]"
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
3,M1G,Scarborough,[Woburn]
4,M1H,Scarborough,[Cedarbrae]
